# Generating global explanations of LLM-as-a-Judge using GloVE algorithm


In [ ]:
from src.risk_atlas_nexus import RiskAtlasNexus
from risk_policy_distillation.datasets.prompt_response_dataset import (
    PromptResponseDataset,
)
from risk_atlas_nexus.blocks.inference import RITSInferenceEngine
from risk_atlas_nexus.blocks.inference.params import RITSInferenceEngineParams

from datasets import load_dataset

#### Create a dataset

To explain the LLM-as-a-Judge you need to provide a dataset. [AbstractDataset](../src/models/datasets/abs_dataset.py) class provides a wrapper for a dataframe you want to explain. You can use [PromptDataset](../src/models/datasets/prompt_dataset.py) or [PromptResponseDataset](../src/models/datasets/prompt_response_dataset.py) depending on whether your dataframe consists of only prompts or prompt-response pairs. You can also create a custom dataset by inheriting the Dataset class.

You have to provide a config with information on column name mapping. Additional parameters: _flip_labels_ indicates whether labels of the dataframe should be flipped in preprocessing step (e.g. for BeaverTails where labels indicate that the content is safe rather than harmful); _split_ indicates whether a train-val-test split needs to be performed during preprocessing.


In [13]:
# get the dataset
ds = load_dataset("PKU-Alignment/BeaverTails")
dataframe = ds.data["330k_train"].table.to_pandas()

# sample for testing purposes
dataframe = dataframe.sample(10)

# provide some information in the data config
data_config = {
    "dataset_name": "BeaverTails",
    "index_col": "",  # if there is no index column it will be automatically populated
    "prompt_col": "prompt",
    "response_col": "response",
    "label_col": "is_safe",
}

dataset = PromptResponseDataset(dataframe=dataframe, config=data_config)

### Create a guardian config


In [ ]:
guardian_config = {
    "task": f"Harm detection",
    "criterion": f"Harm",
    "criterion_definition": f"The message has harmful by common-sense.",
    "labels": [0, 1],
    "label_names": ["harmless", "harmful"],
    "output_labels": ["no", "yes"],
}

##### Policy Distillation uses granite gurdian as a LLM-as-a-Judge, and other Large Language Models (LLMs) to query at various stages in the pipeline. Therefore requires access to inference or call the model.

**Available Inference Engines**: WML, Ollama, vLLM, RITS. Please follow the [Inference APIs](https://github.com/IBM/risk-atlas-nexus?tab=readme-ov-file#install-for-inference-apis) guide before going ahead.

_Note:_ RITS is intended solely for internal IBM use and requires TUNNELALL VPN for access.


In [ ]:
guardian_judge = RITSInferenceEngine(
    model_name_or_path="ibm-granite/granite-guardian-3.3-8b",
    credentials={
        "api_key": "RITS_API_KEY",
        "api_url": "https://inference-3scale-apicast-production.apps.rits.fmaas.res.ibm.com",
    },
    parameters=RITSInferenceEngineParams(
        logprobs=True, top_logprobs=10, temperature=0.0
    ),
)

llm_component = RITSInferenceEngine(
    model_name_or_path="meta-llama/llama-3-3-70b-instruct",
    credentials={
        "api_key": "RITS_API_KEY",
        "api_url": "https://inference-3scale-apicast-production.apps.rits.fmaas.res.ibm.com",
    },
)

### Create and run the explanation generation pipeline

Pipeline streamlines local and global explanation generation process. Extractor executes the CLoVE algorithm and generates a set of local explanations, and Clusterer executes GloVE algorithm and merges the local explanations into a global one.

Pass `lime=False` to pipeline creation step if no local word-based verification is done. SImilarly, use `fr=False` if FactReasoner is not used to verify global explanations.

The resulting local and global explanations are saved in the path folder passed to the pipeline.run() call.
The execution logs can be found in the logs folder.


In [ ]:
risk_atlas_nexus = RiskAtlasNexus()

expl = risk_atlas_nexus.generate_policy_rules(
    guardian_config, dataset, guardian_judge, llm_component, local_expl="LIME"
)

print(expl.print())

[2025-10-28 10:04:01:384] - INFO - RiskAtlasNexus - Created RiskAtlasNexus instance. Base_dir: None
[2025-10-28 10:04:01:912] - INFO - RiskAtlasNexus - Created RITS inference engine.
[2025-10-28 10:04:02:269] - INFO - RiskAtlasNexus - Created RITS inference engine.
0it [00:00, ?it/s]

Inferring with RITS:   0%|          | 0/1 [00:00<?, ?it/s]

Inferring with RITS:   0%|          | 0/100 [00:00<?, ?it/s]

Inferring with RITS:   0%|          | 0/1 [00:00<?, ?it/s]

Inferring with RITS:   0%|          | 0/1 [00:00<?, ?it/s]

...

NLI: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 27060.03prompts/s]


Inferring with RITS:   0%|          | 0/1 [00:00<?, ?it/s]

RITS = True Model = llama-3.1-8b-instruct
[NLIExtractor] Using LLM on RITS: llama-3.1-8b-instruct
[NLIExtractor] Prompt version: v1
[NLIExtractor] Prompts created: 2


NLI: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 17549.39prompts/s]

{'rules': [{'prediction': 0, 'if_clause': 'uses respectful language', 'despite_clauses': 'none'}, {'prediction': 1, 'if_clause': 'contains hurtful social bias', 'despite_clauses': 'none'}, {'prediction': 0, 'if_clause': 'no offensive content', 'despite_clauses': 'none'}, {'prediction': 1, 'if_clause': 'damage information provided', 'despite_clauses': 'none'}, {'prediction': 1, 'if_clause': 'explicit instructions', 'despite_clauses': 'none'}, {'prediction': 1, 'if_clause': 'weapon creation', 'despite_clauses': 'none'}, {'prediction': 1, 'if_clause': 'derogatory language', 'despite_clauses': 'none'}, {'prediction': 1, 'if_clause': 'explicit harm method', 'despite_clauses': 'none'}, {'prediction': 1, 'if_clause': 'physical harm', 'despite_clauses': 'none'}, {'prediction': 1, 'if_clause': 'explicit language', 'despite_clauses': 'none'}, {'prediction': 1, 'if_clause': 'objectifying language', 'despite_clauses': 'none'}, {'prediction': 0, 'if_clause': 'helpful advice', 'despite_clauses': 'no